### Step 0
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

In [202]:
# Import pandas package 
import pandas as pd
# Import numy package
import numpy as np
# Now, we will work with IMDB dataset
data_extraction = pd.ExcelFile('data/Energy Indicators.xls')
energy = data_extraction.parse(skiprows=17,skip_footer=(38),usecols=[1,3,4,5],
                               names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])

# make sure missing data is reflected as np.NaN values.
columns = ['Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy[columns] = energy[columns].replace('...',np.NaN)
# convert Energy Supply to gigajoules
energy.loc['Energy Supply'] = energy['Energy Supply']*(10**6)
# rename the following list of countries
energy['Country'] = energy['Country'].replace({'China, Hong Kong Special Administrative Region':'Hong Kong',
                                               'United Kingdom of Great Britain and Northern Ireland':'United Kingdom',
                                               'Republic of Korea':'South Korea','United States of America':'United States',
                                               'Iran (Islamic Republic of)':'Iran'})
# there are several countries with numbers and/or parenthesis in their name. Be sure to remove them
energy['Country'] = energy['Country'].str.replace(r" \(.*\)","")

print(energy.count())

Country                     240
Energy Supply               222
Energy Supply per Capita    222
% Renewable                 227
dtype: int64


### Step 1
<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

In [203]:
GDP = pd.read_csv('data/world_bank.csv', header = 4)

GDP['Country Name'].replace({"Korea, Rep.":"South Korea", 
                             "Iran, Islamic Rep.":"Iran",
                             "Hong Kong SAR, China": "Hong Kong"}
                            , inplace=True)
print(GDP.count())

Country Name      264
Country Code      264
Indicator Name    264
Indicator Code    264
1960              119
1961              120
1962              120
1963              120
1964              120
1965              127
1966              132
1967              133
1968              137
1969              137
1970              146
1971              146
1972              146
1973              146
1974              147
1975              152
1976              153
1977              158
1978              158
1979              159
1980              172
1981              177
1982              181
1983              181
1984              185
1985              187
1986              190
1987              193
1988              196
1989              199
1990              212
1991              214
1992              217
1993              218
1994              220
1995              226
1996              226
1997              227
1998              228
1999              230
2000              235
2001      

### Step 2
Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

In [204]:
ScimEn = pd.read_excel('data/scimagojr-3.xlsx')
print(ScimEn.count())

Rank                      191
Country                   191
Documents                 191
Citable documents         191
Citations                 191
Self-citations            191
Citations per document    191
H index                   191
dtype: int64


### Step 3
Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This step should yeild a DataFrame with 20 columns and 15 entries.*

In [205]:
# ScimEn
# only the top 15 countries
ScimEn = ScimEn[:15]

# GDP
# only the last 10 years (2006-2015)
GDP = GDP[['Country Name','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]
GDP.columns = ['Country','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']

# Merge ScimEn & energy
df1 = pd.merge(ScimEn, energy, how='inner', left_on='Country', right_on='Country')
# Merge (ScimEn & energy) & GDP
df2 = pd.merge(df1, GDP,how='inner', left_on='Country', right_on='Country')
# set index for the table
df2 = df2.set_index('Country')
print(df2)

                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       889

### Step 4
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This step should yield a single number.*

### Step 5

#### Answer the following questions in the context of only the top 15 countries by Scimagojr Rank 


What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This step should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [206]:
# 15 countries 
# average GDP 
# descending order
data = df2
years = ['2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']
avgGDP = data[years].mean(axis = 1).rename('avgGDP').sort_values(ascending= True)
print(avgGDP)

Country
Iran                  4.441558e+11
South Korea           1.106715e+12
Australia             1.164043e+12
Spain                 1.418078e+12
Russian Federation    1.565459e+12
Canada                1.660647e+12
India                 1.769297e+12
Italy                 2.120175e+12
Brazil                2.189794e+12
United Kingdom        2.487907e+12
France                2.681725e+12
Germany               3.493025e+12
Japan                 5.542208e+12
China                 6.348609e+12
United States         1.536434e+13
Name: avgGDP, dtype: float64


### Step  6
What is the mean `Energy Supply per Capita`?

*This step should return a single number.*

In [207]:
# print(data.iloc[:,8])
meanESPC = data.iloc[:,8].mean()
print(meanESPC)

157.6


### Step 7
What country has the maximum % Renewable and what is the percentage?

*This step should return a tuple with the name of the country and the percentage.*

In [220]:
# idxmax() – > returns the index of the row where column name “% Renewable” has maximum value.
maxRenewable = data.iloc[:,9].max()
country = data['% Renewable'].idxmax()

maxRenewableResult = (country, maxRenewable)
print(result)

('Brazil', 69.64803)


### Step 8
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This step should return a tuple with the name of the country and the ratio.*

In [223]:
citationsRatio = data["Self-citations"] / data["Citations"]

citationsRatioResult = (citationsRatio.idxmax(), citationsRatio.max())
print(citationsRatioResult)

('China', 0.6893126179389422)


### Step 9
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This step should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [228]:
# the median for all countries in the top 15
medianRenewable = data["% Renewable"].median(axis=0)

# 1 if the country's % Renewable value is at or above the median
# 0 if the country's % Renewable value is below the median
data["HighRenew"] = data.apply(lambda c: 1 if c["% Renewable"] > medianRenewable else 0, axis=1)

#sorted in ascending order of rank
data.sort_values(by='Rank', inplace=True)

print(data["HighRenew"])

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64


Step 10
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [229]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

data['Estimate Population'] = data['Energy Supply'] / data['Energy Supply per Capita']

continentGroups = pd.DataFrame(columns = ['size', 'sum', 'mean', 'std'])
# size, sum, mean, and std deviation for the estimated population of each country.
for group, df in data.groupby(ContinentDict):
    continentGroups.loc[group] = [len(df),
                         df['Estimate Population'].sum(),
                         df['Estimate Population'].mean(),
                         df['Estimate Population'].std()]
    
print(continentGroups)

               size          sum        mean         std
Asia            5.0  2898.666387  579.733277  679.097888
Australia       1.0    23.316017   23.316017         NaN
Europe          6.0   457.929667   76.321611   34.647667
North America   2.0   352.855249  176.427625  199.669645
South America   1.0   205.915254  205.915254         NaN
